Relatório do trabalho
==============
**Nome:** Álvaro Leandro Cavalcante Carneiro
**Linguagem utilizada:** Python 3.6

Os códigos e o relatório foram desenvolvidos em um Jupyter notebook e sua versão online pode ser encontrada aqui: https://colab.research.google.com/drive/1HQIBTnHCrcSrfAUOsx7836wZzDnQ9Fhd?usp=sharing. Nessa versão é possível executar novamente os algoritmos.

# Qual o problema ? 
Utilizar uma rede neural no modelo perceptron para identificar 3 classes de flores baseados em suas características de tamanho de pétalada e sépala.

## Importação das bibliotecas
Ferramentas usadas no processo de desenvolvimento

In [1]:
import random
import numpy as np
import pandas as pd
import math

## Análise Exploratória dos Dados (AED)
Vamos entender um pouco mais do nosso dataset, e deixar os dados em um formato que seja favorável para nossa rede neural perceptron.

O primeiro passo foi ler o arquivo baixado aqui: http://archive.ics.uci.edu/ml/datasets/Iris.
O arquivo em questão não possuia colunas, portanto eu modifiquei o mesmo para adicionar o nome das colunas na primeira linha e deixar o arquivo no formato .CSV ao invés do .DATA. 

In [2]:
dataframe = pd.read_csv('/home/alvaro/Documentos/mestrado/computação bio/redes neurais/datasets/iris2.csv', header = 0)

dataframe.head()

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Um passo inicial importante é verificar se o nosso dataset possui inconsistências quanto aos valores, podendo ser algum outlier, ruído, valor vazio, etc...

In [3]:
print('Valores nulos:')
print(dataframe.isna().sum())
dataframe.describe()

Valores nulos:
sepal-length    0
sepal-width     0
petal-length    0
petal-width     0
class           0
dtype: int64


,sepal-length,sepal-width,petal-length,petal-width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## Normalização dos atributos previsores
O método *isnul()* nos mostrou que não há nenhum registro vazio no nosso dataset, além disso, é possível observar que os valores também parecem estar todos coerentes, sem a presença de outliers, como podemos notar pelo desvio padrão e mínimo e máximo de cada coluna.

Todavia, existe uma variação relativamente grande dentro do nosso domínio de atributos previsores. O atributo *petal-width* por exemplo, tem uma média de valor de 1.1, enquanto o *sepal-length* possui uma média de 5.8, portanto se faz necessário a padronização dos valores, para que nosso ajuste dos pesos não seja muito influenciado por essa diferença no tamanho da entrada. 

O tipo de normalização escolhido foi o **Z-score**, de forma arbitrária, por ser bastante comum em problemas como esse. Sua fórmula é bastante simples e foi representada no método *normalizacao_z_score*.

Ainda um pouco antes de fazer a normalização, dividi nosso *dataframe* em uma variável chamada **previsores** e outra chamada **classe**, como o nome sugere, os previsores são as colunas com as características das flores que serão usadas para tentar ajustar nossos pesos da rede de maneira à generalizar uma solução que encontre as classes corretamente.

In [4]:
previsores = dataframe.iloc[:, 0:4] 
classe = dataframe['class']

In [5]:
def normalizacao_z_score(valor):
    media = previsores[valor.name].mean()
    desvio_padrao = previsores[valor.name].std()

    return (valor - media) / desvio_padrao

O método *apply()* com a **lambda** aplicam o processo matemático do nosso método de normalização em cada um dos registros do *dataframe*.

In [6]:
previsores = previsores.apply(lambda row: normalizacao_z_score(row))
previsores.head()

,sepal-length,sepal-width,petal-length,petal-width
0,-0.897674,1.028611,-1.336794,-1.308593
1,-1.139200,-0.124540,-1.336794,-1.308593
2,-1.380727,0.336720,-1.393470,-1.308593
3,-1.501490,0.106090,-1.280118,-1.308593
4,-1.018437,1.259242,-1.336794,-1.308593


## Tratando valores categóricos
Podemos ver agora que nossos valores estão em uma escala completamente diferente, todavia isso os torna mais coerente de serem trabalhados.

O próximo passo será transformar o valor da nossa classe de categórico para discreto, para que seja possível calcular o erro da saída por exemplo.

O primeiro passo para isso foi criar um método que gera uma estrutura de dicionário dinâmica, baseado na quantidade de classes do problema que está sendo tratado. Para isso, basta percorrer a classe existente e atribuir um valor inteiro para cada classe.

In [7]:
def get_dicionario_classes(classe):
    dict_classes = {}
    count = 0
    
    for i in classe.unique():
        dict_classes[i] = count
        count += 1
        
    return dict_classes

In [8]:
dict_classes = get_dicionario_classes(classe)
print(dict_classes)

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


Podemos ver acima os valores que nosso método atribuiu para cada uma das classes.

Basta agora repetir o processos anterior de usar o método *apply()*, porém agora passando no *lambda* o método que vai atribuir a classe a seu determinado valor no dicionário que criamos anteriormente.

In [9]:
def transformar_categorico_em_numerico(valor, dict_classes):
    return dict_classes[valor]
    
classe = classe.apply(lambda row: transformar_categorico_em_numerico(row, dict_classes))
print('Classe no formato numérico', classe.value_counts())

Classe no formato numérico 2    50
1    50
0    50
Name: class, dtype: int64


## Lidando com problemas multi-classe
O problema em questão é multi-classe, ou seja, possuímos mais de duas classes como resposta na nossa camada de saída, podendo ser, íris-setosa, versicolor e virginica.
Para problemas binários utilizar um único neurônio com a saída de 0 e 1 nos basta, todavia aqui, vamos precisar criar um novo neurônio para cada classe, totalizando 3 na nossa camada de saída.

Além de modificar a estrutura da rede neural, também vamos precisar codificar nossos valores, uma vez que, ao invés de um valor escalar vamos trabalhar com um array na saída da rede, sendo ele representado por: [1,0,0], [0,1,0] e [0,0,1].

In [10]:
def codificar_classe():
    classe_codificada = {}
    array_classe = [1] + [0] * (len(classe.unique()) - 1) #cria um array dinâmico baseado na 
    #quantidade de classes, é [1,0,0] para esse problema mas poderia ser [1,0,0,0....,0].
    count = 1    
    classe_codificada[0] = array_classe.copy()
    
    for i in range(len(classe.unique()) - 1):
        array_classe[count - 1] = 0
        array_classe[count] = 1     
        classe_codificada[count] = array_classe.copy()
        count += 1
    
    return classe_codificada      

classe_codificada = codificar_classe()

In [11]:
classe_codificada

{0: [1, 0, 0], 1: [0, 1, 0], 2: [0, 0, 1]}

A ideia do método *codificar_classe* é criar mais um dicionário, como podemos ver acima, onde cada posição representa uma classe codificada em um array de binários. O tamanho desse array é dinâmico dependendo do número de classes e a ideia é ir movimentando o valor do 1 conforme as iterações.

Feito isso, basta repetir o processo para substituir o valor da classe.

In [12]:
def substituir_classe_codificada(valor, classe_codificada):
    return classe_codificada[valor]

classe = classe.apply(lambda row: substituir_classe_codificada(row, classe_codificada))
print(classe.head())

0    [1, 0, 0]
1    [1, 0, 0]
2    [1, 0, 0]
3    [1, 0, 0]
4    [1, 0, 0]
Name: class, dtype: object


Com isso, a classe agora está em uma estrutura que vai suportar o problema multi-classe.

## Inicialização dos pesos
Os pesos serão inicializados de forma aleatória para então serem gradativamente ajustados conforme a rede neural converge.
Para isso, foi criado o método *inicializar_pesos*, que percorre cada um dos neurônios e gera um vetor da quantidade de pesos que ele possui baseado nas suas conexões sinapticas com os neurônios da próxima camada.

Além disso, nosso método também recebe um parâmetro chamado dominio, que é o intervalo de valores que os pesos serão gerados, os testes a princípio foram realizados em um domínio de [0,1].

In [29]:
def inicializar_pesos(dominio):
    pesos_final = []
    
    for i in range(len(previsores.columns)):
        pesos = [] 
        for j in range(len(dict_classes)):
            pesos.append(random.uniform(dominio[0], dominio[1]))
        pesos_final.append(pesos)
    return pesos_final

In [14]:
pesos = inicializar_pesos()
print('Pesos:', pesos)

Pesos: [[0.2792888346935375, 0.650936129351441, 0.5877349714324628], [0.06968998177646468, 0.521682714640433, 0.2283172562238035], [0.12142492407255268, 0.604884070630378, 0.7739583379263297], [0.12064904021797795, 0.3598282526162032, 0.5248123922348522]]


Como podemos ver acima, nosso array de pesos para esse problema possui 4 posições com 3 pesos em cada uma das posições, isso acontece por que possuímos 4 neurônios (nosso atributos de entrada) conectados à 3 neurônios (um neurônio para cada saída possível), portanto cada um dos 4 neurônios possui 3 pesos (conexões) cada um. 

Para obter o número de conexões por camada basta multiplicar o número de neurônios da camada atual pelos número de neurônios na próxima camada, dessa forma temos: 4 * 3 = 12.

## Implementação da função de soma
O perceptron, assim como o perceptron multicamadas, possui diversas operações que são realizadas entre os neurônios, esses operações foram representadas em diferentes métodos peqenos.
O primeiro deles é a função de soma, que acontece em todos os neurônios, somando o valor do produto da multiplicação entre o neurônio adjacente anterior com o peso da sinapse artificial.
Isso foi feito utilizando a biblioteca do numpy por uma questão de performance, onde a função dot nos retorna o produto escalar que desejamos.

In [15]:
def somatoria(entradas, pesos):
    return np.dot(entradas, pesos) 

## Função de ativação
A função de ativação por default no perceptron é a chamada "step function (função degrau)", onde o neurônio artificial é excitado ou não baseado em um treshold (limiar) pré definido.
Nesse caso, se o valor do neurônio for maior que 0 ele retorna o 1, excitando a célula, caso contrário, retorna 0.

Por ser um problema multiclasse, foi criado um *for* para percorrer o array da classe e excitar todas as posições em que o valor é maior que 0. 

Claro que isso acaba gerando um problema onde mais de um neurônio por vez na camada de saída é excitado. 

In [16]:
def funcao_ativacao_step(soma):
    ativacao = []
    for i in soma:
        if i > 0:
            ativacao.append(1)
        else:
            ativacao.append(0)

    return ativacao

## Função de custo
A função de custo é a responsável por calcular o erro da nossa rede neural, ao comparar o valor correto com o valor que foi previsto.

In [17]:
def funcao_custo(valor_correto, valor_previsto):
    erro = list(abs(np.array(valor_correto) - np.array(valor_previsto)))
    return sum(erro) # valor escalar

## Função de atualização de peso
A atualização de pesos no perceptron é definida pela seguinte formula:


Com isso, conseguimos ajustar os pesos seguindo um taxa de aprendizado, basicamente o tamanho do nosso "passo", além de levar em consideração a grandeza da entrada e o quanto a previsão estava incorreta.

Algo importante de se considerar é que o perceptron faz a atualização dos pesos POR REGISTRO, ou seja, a cada registro apresentado à rede neural que é classificado de forma incorreta gera uma atualização nos pesos, podendo dificultar a generalização dos pesos devido à sensibilidade aos dados contidos no dataset.

In [18]:
def atualizar_peso(entrada, peso, erro, tx_aprendizado = 0.001):
    novo_peso = peso + (tx_aprendizado * entrada * erro)
    return novo_peso

## Implementação do perceptron
A função de treinamento recebe o número de épocas que iremos executar nosso perceptron, onde cada época representa a passagem de todo o dataset na nossa rede. A estratégia seguida a princípio para atualização dos pesos é por registro, ou seja, para cada registro errado nós fazemos os ajustes nos pesos. Geralmente no perceptron multicamadas essa atualização é feita por épocas ou batches.

In [19]:
def treinar(epocas, f_ativacao):
    execucoes = 0
    precisoes = [0]
    while execucoes < epocas:
        precisao = 0
        iteracao = 0

        np.random.shuffle(previsores.values) # embaralhar os valores dos previsores, por que sem isso, podemos ter sempre uma ordem fixa de ajuste de pesos, prejudicando a rede

        for i in previsores.values:
            entradas = i   
            soma = somatoria(entradas, pesos)
        
            ativacao = f_ativacao(soma)
        
            erro = funcao_custo(classe[iteracao], ativacao) # baseado no meu resultado previsto, dado na última função de ativação.
        
            if erro > 0:
                count = 0

                for i in entradas:
                    novo_peso = atualizar_peso(i, pesos[count], erro)
                    pesos[count] = novo_peso
                    count += 1
            else:
                precisao += 100 / len(previsores)
                precisoes.append(precisao)

            iteracao += 1
        
        execucoes += 1
    return max(precisoes)

## Testes no perceptron
Uma vez que a rede perceptron foi criada só nós resta testar. Para isso, o algoritmo foi inicializado 50 vezes, sem nenhuma mudança nos dados, apenas gerando um conjunto inicial de pesos diferente para cada inicialização.

As redes neurais são algoritmos extremamente estocásticos, o que faz com que sejam inclusive evitados em algumas áreas, portanto é interessante tentar treinar a rede várias vezes para tentar capturar seus melhores resultados.

In [24]:
precisao_rede = []
for i in range(50):
    pesos = inicializar_pesos([0, 1]) #iniciamos novamente os pesos para mudá-los em cada teste
    precisao_rede.append(treinar(100, funcao_ativacao_step))

print('Melhor precisão da rede', max(precisao_rede))

Melhor precisão da rede 12.666666666666663


### Resultados iniciais e ZeroR
Nossos testes inicias nos mostram uma precisão extremamente baixa, algo está definitivamente se comportando mal no algoritmo. Podemos ter certeza que o culpado é nosso algoritmo e não a base de dados devido à análise exploratória nos dados que foi feita anteriormente, além de do fato de ter o conhecimento de que é um problema linearmente separável.

Para saber a precisão mínima que um algoritmo deve ter para ser considerado melhor do que não usar algoritmo algum ao fazer previsões é o "zeroR". Basicamente se classificarmos todos os nosso registros como pertencendo à classe majoritária, qual será nossa precisão?


Nesse dataset as classes estão **normalmente distribuídas** em 3 partes iguais, fazendo com que nossa precisão majoritária sem algoritmos seja de **33%**.


Todavia ainda que nosso algoritmo seja considerado útil com 34% de acerto, não faz sentido investir em algo com números tão baixos.

### Função de ativação sigmoid.
Como foi dito anteriormente, utilizar a função degrau para ativação pode gerar no aumento dos erros, devido a possibilidade de excitar vários neurônios ao passar pelo limiar especificado.

Portanto foi criado uma nova função de ativação baseado na sigmoid, definida pela seguinte fórmula:


Com isso, nossos valores ficam em um intervalo de 0 e 1 e o neurônio escolhido para ser excitado é aquele com o maior valor. Dessa forma, já não temos mais o problema de excitar vários neurônios. 

In [25]:
def funcao_ativacao_sigmoid(soma):
    resultado = list(1 / (1 + math.e ** -soma))
    index_excitacao = resultado.index(max(resultado)) 
    resultado = [0] * len(soma)
    resultado[index_excitacao] = 1
    
    return resultado

In [26]:
precisao_rede = []
for i in range(50):
    pesos = inicializar_pesos([0, 1]) #iniciamos novamente os pesos para mudá-los em cada teste
    precisao_rede.append(treinar(100, funcao_ativacao_sigmoid))

print('Melhor precisão da rede', max(precisao_rede))

Melhor precisão da rede 38.666666666666664


Conseguimos melhorar consideravelmente os resultados com essa nova função de ativação.

### Bias
Algo que podemos fazer para melhorar a generalização da rede é adicionar o Bias, sendo uma constante que será considerada como um neurônio na camada de entrada.

In [28]:
previsores['bias'] = 1
precisao_rede = []
for i in range(50):
    pesos = inicializar_pesos([0, 1]) #iniciamos novamente os pesos para mudá-los em cada teste
    precisao_rede.append(treinar(100, funcao_ativacao_sigmoid))

print('Melhor precisão da rede', max(precisao_rede))

Melhor precisão da rede 37.333333333333336


Como podemos ver, a aplicação do bias também nos ajudo a melhorar os resultados da rede, chegando em torno de 70% de precisão.